In [5]:
using Primes, Combinatorics

In [2]:
p = 77432081

77432081

In [8]:
function test_solovay_strassen(a, n, salida = false)
	if salida
		println("Test de Solovay Strassen para a = ", a, ", n = ", n)
	end

	powmod = powermod(a, div(n-1, 2), n)
	
	if powmod ∉ [1, n-1]
		if salida
			println("\ta^{(n-1)/2} mod n = ", powmod, ", que no es congruente con 1 o -1. Falla.")
		end
		return false
	end

	if salida
		println("\ta^{(n-1)/2} mod n = ", powmod, ". Jacobi = ", jacobisymbol(a, n), "\n")
	end

	return mod(jacobisymbol(a, n), n) == powmod
end

function lucas_lehmer(n)
	factores = collect(keys(factor(n-1)))

	for a in 1:(n-1)

		if powermod(a, n-1, n) == 1
			resultado = map(q -> powermod(a, div(n-1, q), n), factores)
			if 1 ∉ resultado
				return true
			else
				posicion = first(findall(x -> x == 1, resultado))
			end
		end
	end

	return false
end

function ρ_Polard(n, f, t = 1000, x0 = 2)
	# En caso en el que sea divisible por dos, es posible que falle, así que lo comprobamos a mano.
	if mod(n, 2) == 0
		return [[2, div(n, 2)], 0]
	end

	x = x0
	y = x
	i = 0

	while i < t
		i = i + 1
		x = mod(f(x), n)
		y = mod(f(f(y)), n)
		g = gcd(x - y, n)

		if 1 < g && g < n
			return [[g, div(n, g)], i]
		end
	end

	return []
end

function descomponer(n)
	por_descomponer = [n]
	irreducibles    = []
	
	f     = x -> x^2 + 1
	f_alt = x -> x^2 - 1
	
	i = 0

	while length(por_descomponer) > 0
		println("\nPor descomponer: $por_descomponer")
		num = pop!(por_descomponer)

		# Aplica el test de Miller Rabin primero. Si sale posible primo, saca 
		# un certificado de primalidad con Lucas Lehmer.
		if isprime(num) && lucas_lehmer(num)
			println("\t-> $num es primo")
			push!(irreducibles, num)
		else
			println("\t-> Descomponiendo $num")
			resultado = ρ_Polard(num, f, 3000)

			# Dado que ρ_Polard puede fallar, probamos con otra función alternativa. 
			# Por ejemplo, para 25, x -> x^2 + 1, no se devuelve nada. 
			
			if length(resultado) == 0
				resultado = ρ_Polard(num, x -> x^2 - 1, 4000)
			end
			
			i = i + resultado[2]
			por_descomponer = vcat(por_descomponer, resultado[1])
		end
	end

	return [sort(irreducibles), i]
end

descomponer (generic function with 1 method)

In [9]:
primos = [2, 3, 5, 7, 9, 11, 13, 17]
map(a -> test_solovay_strassen(a, p), primos)

8-element Vector{Bool}:
 1
 1
 1
 1
 1
 1
 1
 1

In [6]:
lucas_lehmer(p)

true

In [10]:
descomponer(p-1)


Por descomponer: [77432080]
	-> Descomponiendo 77432080

Por descomponer: [2, 38716040]
	-> Descomponiendo 38716040

Por descomponer: [2, 2, 19358020]
	-> Descomponiendo 19358020

Por descomponer: [2, 2, 2, 9679010]
	-> Descomponiendo 9679010

Por descomponer: [2, 2, 2, 2, 4839505]
	-> Descomponiendo 4839505

Por descomponer: [2, 2, 2, 2, 11, 439955]
	-> Descomponiendo 439955

Por descomponer: [2, 2, 2, 2, 11, 5, 87991]
	-> 87991 es primo

Por descomponer: [2, 2, 2, 2, 11, 5]
	-> 5 es primo

Por descomponer: [2, 2, 2, 2, 11]
	-> 11 es primo

Por descomponer: [2, 2, 2, 2]
	-> 2 es primo

Por descomponer: [2, 2, 2]
	-> 2 es primo

Por descomponer: [2, 2]
	-> 2 es primo

Por descomponer: [2]
	-> 2 es primo


2-element Vector{Any}:
  Any[2, 2, 2, 2, 5, 11, 87991]
 5